In [2]:
!pip install vllm -q
!pip install bitblas>=0.1.0 -q
!pip install optimum -q

In [3]:
!pip uninstall torch -y -q
!pip install torch==2.6 -q
!pip install gptqmodel -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gptqmodel 2.2.0 requires protobuf>=5.29.3, but you have protobuf 4.25.7 which is incompatible.
fastai 2.7.18 requires torch<2.6,>=1.10, but you have torch 2.6.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.26.0 requires protobuf<5.0,>=3.19, but you have protobuf 6.30.2 which is incompatible.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 6.30.2 which is incompatible.
google-api-core 1.34.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<4.0.0dev,>=3.19.5, but you have protobuf 6.30.2 which

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "RedHatAI/Meta-Llama-3.1-8B-Instruct-quantized.w4a16"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

2025-05-03 21:05:56.382485: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746306356.406519     537 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746306356.413915     537 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'


INFO  ENV: Auto setting PYTORCH_CUDA_ALLOC_CONF='expandable_segments:True' for memory saving.      
INFO  ENV: Auto setting CUDA_DEVICE_ORDER=PCI_BUS_ID for correctness.                              
INFO   Kernel: Auto-selection: adding candidate `ExllamaQuantLinear`                               


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Some weights of the model checkpoint at RedHatAI/Meta-Llama-3.1-8B-Instruct-quantized.w4a16 were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.10.sel

INFO  Format: Converting `checkpoint_format` from `gptq` to internal `gptq_v2`.                    
INFO  Format: Converting GPTQ v1 to v2                                                             
INFO  Format: Conversion complete: 0.01946258544921875s                                            


In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('/kaggle/input/essays/data.csv')

In [7]:
essays = []
# good
for row, column in df[df['Overall'] == 5.0].sample(4, random_state=42).iterrows():
  essays.append(column['full_text'].replace('\n\n', ' ').strip())
for row, column in df[df['Overall'] == 3.0].sample(3, random_state=42).iterrows():
  essays.append(column['full_text'].replace('\n\n', ' ').strip())
for row, column in df[df['Overall'] == 1.0].sample(3, random_state=42).iterrows():
  essays.append(column['full_text'].replace('\n\n', ' ').strip())

In [8]:
def get_feedback(essay, temperature):
  prompt = f"""You are given the following essay.

Provide feedback in a structured and concise format, addressing ONLY the following aspects:
- Overall
- Cohesion
- Syntax
- Vocabulary
- Phraseology
- Grammar and Conventions

For each aspect:
- Explicitly note possible improvements if applicable
- DO provide a general impression or summary inside overall aspect part.

{essay}
"""

  messages = [
    {"role": "system", "content": "You are a helpful English language teaching assistant."},
    {"role": "user", "content": prompt},
]

  text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    temperature=temperature
)

  model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

  outputs = model.generate(
    **model_inputs,
    max_new_tokens=2048
)

  generated_ids = outputs[0][model_inputs.input_ids.shape[-1]:]

  response = tokenizer.decode(generated_ids, skip_special_tokens=True)

  return response

In [9]:
feedbacks = []
for essay in essays:
    feedbacks.append(get_feedback(essay, 0.5))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [10]:
with open('feedbacks.txt', 'w', encoding='utf-8') as f:
  for i in range(len(feedbacks)):
    f.write(essays[i][:50] + '\n' + feedbacks[i] + '\n\n\n\n\n')